In [1]:
from keras.models import load_model

In [26]:
model = load_model("./trained_model_gpu.hdf5")

## Predictions

In [3]:
import pickle
with open("notes", "rb") as f:
    notes = pickle.load(f)

In [4]:
pitchnames = sorted(set(notes)) # CLASSES

# MAPPING
ele_to_int = dict( (ele, num) for num, ele in enumerate(pitchnames) )
int_to_ele = dict( (num, ele) for num, ele in enumerate(pitchnames) )

In [6]:
# The Training data
sequence_length = 100
network_input = []
n_vocab = len(set(notes))
for i in range(len(notes) - sequence_length):
    
    seq_in = notes[i : i + sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

## How to generate music? 
1. Give any random training point from the training data to the model
2. Discard the 1st element from the training point and append the output
3. Repeat 1 and 2 till the required length of time

In [27]:
import numpy as np

# Random input
start = np.random.randint(len(network_input) - 1)

# Initial Pattern
pattern = network_input[start]
prediction_output = []

# Let's do it for 200 elements
for note_index in range(200):
    
    # (Batch_size X length of the pattern X 1)
    prediction_input = np.reshape(pattern, [1, len(pattern), 1])
    
    # Normalization
    prediction_input = prediction_input/float(n_vocab)
    
    # Prediction : Prob for all classes
    prediction = model.predict(prediction_input, verbose=0)
    # Class number
    idx = np.argmax(prediction)
    
    # Integer to Note
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    # changing the Input Sequence
    pattern.append(idx)
    pattern = pattern[1:]

In [28]:
print(prediction_output)

['7+9+0', '7+9+0', 'F1', 'F#7', '9+11+3', '9+11+2+5', 'C#5', 'C#4', 'B4', 'B3', '9+11+3', '9+11+2+5', '9+11', 'C#4', '7+9', '7+9', '3+7+9', '3+7+9', '11+0', '11+0', '1+2', '1+2', '2+7', '2+7', '3+7+9', '1+4+6+9', '6', '6', 'E-4', 'A0', 'D4', 'D5', '9+10+2+4', 'C6', '9+11+0+4', 'D5', '4+8', 'B-5', 'F#6', 'A4', 'F#7', '11+4', 'D5', 'E-4', '9+10+2+4', '9+11+0+4', 'D5', '4+8', 'C6', 'D5', 'F#6', '8+9+2', '11+4', '11+4', '9+10+2+4', '3+7+9', '4+8', '9+11+0+4', 'E-4', 'F#6', 'D5', '11+4', 'C6', '1+4+6+9', '9+10+2+4', '4+8', 'F#6', '11+4', '9+10+2+4', '4+8', 'F#6', '11+4', '9+10+2+4', '4+8', 'F#6', '11+4', 'D3', '9+11', 'C#3', 'F#2', '9+11', '9+1+4', 'E-2', '4+7+10', 'D2', 'E-2', '11+2+4', 'D3', 'E-2', 'E-1', 'C#4', 'B-3', 'D3', 'D2', 'D3', '9+11', '11+2+4', 'F#2', 'D3', '9+11', 'C#3', 'F#2', '9+11', '9+1+4', 'E-2', '4+7+10', 'B-2', 'E-2', '11+2+4+6', 'D3', 'E-2', 'C#3', 'C3', '11+2+4', 'A1', '9+11', '9+10+2+4', '9+10+2+4', '3+7+9', '9+10+2+4', 'E-1', '9+10+2+4', '9+10+2+4', '9+11', '9+10+2+4

# Create Midi file

In [29]:
from music21 import note, chord, instrument, stream

In [36]:
# Note Object
print(prediction_output[2])
sample_note = note.Note(prediction_output[2])
sample_note.offset = 0
sample_note.instrument = instrument.Piano()

F1


In [37]:
sample_note

<music21.note.Note F>

In [38]:
offset = 0
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        # Notes inside a chord
        temp_notes = []
        for curr_note in notes_in_chord:
            
            # Create a Note object
            new_note = note.Note(int(curr_note))
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
        
        # Creating the cord Object
        new_chord = chord.Chord(temp_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
        
    # if the pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.instrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5 # fixed

In [39]:
output_notes

[<music21.chord.Chord G A C>,
 <music21.chord.Chord G A C>,
 <music21.note.Note F>,
 <music21.note.Note F#>,
 <music21.chord.Chord A B E->,
 <music21.chord.Chord A B D F>,
 <music21.note.Note C#>,
 <music21.note.Note C#>,
 <music21.note.Note B>,
 <music21.note.Note B>,
 <music21.chord.Chord A B E->,
 <music21.chord.Chord A B D F>,
 <music21.chord.Chord A B>,
 <music21.note.Note C#>,
 <music21.chord.Chord G A>,
 <music21.chord.Chord G A>,
 <music21.chord.Chord E- G A>,
 <music21.chord.Chord E- G A>,
 <music21.chord.Chord B C>,
 <music21.chord.Chord B C>,
 <music21.chord.Chord C# D>,
 <music21.chord.Chord C# D>,
 <music21.chord.Chord D G>,
 <music21.chord.Chord D G>,
 <music21.chord.Chord E- G A>,
 <music21.chord.Chord C# E F# A>,
 <music21.chord.Chord F#>,
 <music21.chord.Chord F#>,
 <music21.note.Note E->,
 <music21.note.Note A>,
 <music21.note.Note D>,
 <music21.note.Note D>,
 <music21.chord.Chord A B- D E>,
 <music21.note.Note C>,
 <music21.chord.Chord A B C E>,
 <music21.note.Note D

## Playing the file

In [40]:
midi_stream = stream.Stream(output_notes)

# To write the file on disk
# type, filenam
midi_stream.write('midi', fp='sample.mid')

'sample.mid'

In [41]:
midi_stream.show('midi')

## Drawbacks
- Model doesn't know how to start and end the music ( in terms of melody and all )
- offset is fixed